In [2]:
%pip install --upgrade pip
%pip install ipython-sql prettytable pandas sqlalchemy==1.3.9


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load the SQL magic
%load_ext sql

import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

import pandas as pd

# Create SQLite DB file locally
con = sqlite3.connect("my_data1.db")

# Load the CSV into pandas
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Write to a raw table
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

# Point %sql magic to your SQLite file
%sql sqlite:///my_data1.db


In [4]:
%%sql
DROP TABLE IF EXISTS SPACEXTABLE;

CREATE TABLE SPACEXTABLE AS
SELECT * FROM SPACEXTBL
WHERE Date IS NOT NULL;


 * sqlite:///my_data1.db
Done.
Done.


[]

In [5]:
%%sql
SELECT DISTINCT Launch_Site FROM SPACEXTABLE;


 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [6]:
%%sql
SELECT *
FROM SPACEXTABLE
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5;


 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [7]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass
FROM SPACEXTABLE
WHERE Customer = 'NASA (CRS)';


 * sqlite:///my_data1.db
Done.


Total_Payload_Mass
45596


In [8]:
%%sql
SELECT AVG(PAYLOAD_MASS__KG_) AS Avg_Payload_Mass
FROM SPACEXTABLE
WHERE Booster_Version = 'F9 v1.1';


 * sqlite:///my_data1.db
Done.


Avg_Payload_Mass
2928.4


In [9]:
%%sql
SELECT MIN(Date) AS First_Success_Ground_Pad
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (ground pad)';


 * sqlite:///my_data1.db
Done.


First_Success_Ground_Pad
2015-12-22


In [10]:
%%sql
SELECT DISTINCT Booster_Version
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Success (drone ship)'
  AND PAYLOAD_MASS__KG_ BETWEEN 4000 AND 6000;


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


In [11]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
Success,38
No attempt,21
Success (drone ship),14
Success (ground pad),9
Failure (drone ship),5
Controlled (ocean),5
Failure,3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


In [12]:
%%sql
SELECT Booster_Version, PAYLOAD_MASS__KG_
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);


 * sqlite:///my_data1.db
Done.


Booster_Version,PAYLOAD_MASS__KG_
F9 B5 B1048.4,15600
F9 B5 B1049.4,15600
F9 B5 B1051.3,15600
F9 B5 B1056.4,15600
F9 B5 B1048.5,15600
F9 B5 B1051.4,15600
F9 B5 B1049.5,15600
F9 B5 B1060.2,15600
F9 B5 B1058.3,15600
F9 B5 B1051.6,15600


In [13]:
%%sql
SELECT substr(Date, 6, 2) AS Month,
       Booster_Version,
       Launch_Site,
       Landing_Outcome
FROM SPACEXTABLE
WHERE Landing_Outcome = 'Failure (drone ship)'
  AND substr(Date, 1, 4) = '2015';


 * sqlite:///my_data1.db
Done.


Month,Booster_Version,Launch_Site,Landing_Outcome
01,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
04,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


In [14]:
%%sql
SELECT Landing_Outcome, COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY Landing_Outcome
ORDER BY Outcome_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Outcome_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


In [15]:
%%sql
SELECT Launch_Site 
FROM SPACEXTABLE 
WHERE Launch_Site LIKE 'CCA%' 
LIMIT 5;


 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40
CCAFS LC-40


In [16]:
%%sql
SELECT SUM(PAYLOAD_MASS__KG_) AS Total_Payload_Mass
FROM SPACEXTABLE
WHERE Customer LIKE '%NASA%';


 * sqlite:///my_data1.db
Done.


Total_Payload_Mass
107010


In [18]:
%%sql
SELECT TRIM(Mission_Outcome) AS Mission_Outcome, COUNT(*) AS Total_Count
FROM SPACEXTABLE
GROUP BY TRIM(Mission_Outcome);



 * sqlite:///my_data1.db
Done.


Mission_Outcome,Total_Count
Failure (in flight),1
Success,99
Success (payload status unclear),1


In [20]:
%%sql
SELECT
  SUM(CASE WHEN LOWER(TRIM(Mission_Outcome)) = 'success' THEN 1 ELSE 0 END) AS Success,
  SUM(CASE WHEN LOWER(TRIM(Mission_Outcome)) LIKE 'failure%' THEN 1 ELSE 0 END) AS Failure
FROM SPACEXTABLE;


 * sqlite:///my_data1.db
Done.


Success,Failure
99,1


In [22]:
%%sql
SELECT DISTINCT Booster_Version
FROM SPACEXTABLE
WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);


 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


In [23]:
%%sql
-- Count & rank landing outcomes between 2010-06-04 and 2017-03-20
WITH counts AS (
  SELECT
    TRIM(Landing_Outcome) AS Landing_Outcome,
    COUNT(*) AS Total_Count
  FROM SPACEXTABLE
  WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
  GROUP BY TRIM(Landing_Outcome)
)
SELECT
  Landing_Outcome,
  Total_Count,
  DENSE_RANK() OVER (ORDER BY Total_Count DESC) AS Rank
FROM counts
ORDER BY Total_Count DESC;



 * sqlite:///my_data1.db
Done.


Landing_Outcome,Total_Count,Rank
No attempt,10,1
Failure (drone ship),5,2
Success (drone ship),5,2
Controlled (ocean),3,3
Success (ground pad),3,3
Failure (parachute),2,4
Uncontrolled (ocean),2,4
Precluded (drone ship),1,5


In [24]:
%%sql
SELECT TRIM(Landing_Outcome) AS Landing_Outcome, COUNT(*) AS Total_Count
FROM SPACEXTABLE
WHERE Date BETWEEN '2010-06-04' AND '2017-03-20'
GROUP BY TRIM(Landing_Outcome)
ORDER BY Total_Count DESC;


 * sqlite:///my_data1.db
Done.


Landing_Outcome,Total_Count
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1
